# TRT KPI: Minimize Time to Science
### IN PROGRESS: Updated in PI 22.2 (Spring 2022) 

The following KPIs are measured in this notebook:

* Number of lines of code for a given workflow in the cloud / Number of lines for same workflow on-prem
* Run time for a given workflow in the cloud / run time for same workflow on-prem

#### Workflows: 
* Basic data search, customize, access, plotting, with a collection that is available on-prem and in Earthdata Cloud.

* Data tested (updated in PI 22.2):
    * Using Cloud Metrics to select popular service combination:
        * Synchronous, L2SS, NetCDF, Spatial & variable request
            * Can we filter by non-admin and by collection requested?
    * Working off of [Openscapes OPeNDAP tutorial](https://nasa-openscapes.github.io/2021-Cloud-Hackathon/tutorials/06_S6_OPeNDAP_Access_Gridding.html)
    * Switching from UAT to PROD examples

```
ShortName = 'JASON_CS_S6A_L2_ALT_LR_RED_OST_NRT_F'
concept_id = 'C1968980576-POCLOUD'
```

* On Prem subsetting:
    * [PO.DAAC python script: Subset L1/L2 using OPeNDAP](https://podaac-tools.jpl.nasa.gov/drive/files/common/sw/subsetting/subset_dataset_l2.py)

## Import Packages

In [22]:

#cloud workflow
import requests
from pprint import pprint
from harmony import BBox, Client, Collection, Request, LinkType
from harmony.config import Environment
import datetime as dt

## On-prem Workflow

## Cloud Workflow

In [1]:
# gather times to add up end to end
cloud_times = []

### 1. Determine subsetting service options

#### Steps Involved (NEED TO UPDATE):

* Add step to identify collection of interest

```
Here we are jumping ahead and already know the `concept_id` we are interested in, by browsing cloud-hosted datasets from PO.DAAC in Earthdata Search: https://search.earthdata.nasa.gov/portal/podaac-cloud/search. 

We are going to focus on `MODIS_A-JPL-L2P-v2019.0`: GHRSST Level 2P Global Sea Surface Skin Temperature from the Moderate Resolution Imaging Spectroradiometer (MODIS) on the NASA Aqua satellite (GDS2). Let's first save this as a variable that we can use later on once we request data from Harmony.

We will view the top-level metadata for this collection to see what additional service and variable metadata exist.

We need to isolate the services returned for this dataset:

Inspect the first service returned. Now we're going to search the services endpoint to view that individual service's metadata, like we did with our dataset above. This time, we're explicitly setting the format of the response to umm-json in the Accept Header in order to see detailed metadata about the service.

```

1. Set the token as a request parameter
1. Set desired CMR output format for collections (using the default requires more steps to get at the desired information)
1. Perform CMR collection search using the collection-related search fields
1. Parse the search result
1. Retrieve collection from the search result
1. Retrieve associated service IDs from the collection
1. Set desired (different) output format for services
1. Format a query to fetch the service for each service ID (why isn't this a link relation?)
1. Perform CMR service search for each each service ID
1. Parse the search result
1. Retrieve service from the search result
1. Retrieve associated service options from the service


In [17]:
%time 

url = 'https://cmr.earthdata.nasa.gov/search'
collection_url = f'{url}/{"collections"}'

short_name= 'MODIS_A-JPL-L2P-v2019.0'
concept_id = 'C1940473819-POCLOUD'

response = requests.get(collection_url, 
                        params={
                            'concept_id': concept_id,
                            },
                        headers={
                            'Accept': 'application/json'
                            }
                       )
response = response.json()
# pprint(response)

services = response['feed']['entry'][0]['associations']['services']
print(services)

service_url = "https://cmr.earthdata.nasa.gov/search/services"

service_response = requests.get(service_url, 
                        params={
                            'concept_id': services[0],
                            },
                        headers={
                            'Accept': 'application/vnd.nasa.cmr.umm_results+json'
                            }
                       )
service_response = service_response.json()
pprint(service_response)

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 5.25 µs
['S1962070864-POCLOUD', 'S2004184019-POCLOUD', 'S2153799015-POCLOUD', 'S2227193226-POCLOUD']
{'hits': 1,
 'items': [{'meta': {'concept-id': 'S1962070864-POCLOUD',
                     'concept-type': 'service',
                     'deleted': False,
                     'format': 'application/vnd.nasa.cmr.umm+json',
                     'native-id': 'POCLOUD_podaac_l2_cloud_subsetter',
                     'provider-id': 'POCLOUD',
                     'revision-date': '2022-05-31T23:46:37.054Z',
                     'revision-id': 25,
                     'user-id': 'chen5510'},
            'umm': {'AccessConstraints': 'None',
                    'Description': 'Endpoint for subsetting L2 Subsetter via '
                                   'Harmony',
                    'LongName': 'PODAAC Level 2 Cloud Subsetter',
                    'MetadataSpecification': {'Name': 'UMM-S',
                                              

### 2. Determine variables for collection of interest

#### Steps Involved (NEED TO UPDATE):

```
Several variable records are returned. Again, like we did for services, we'll search the variables endpoint to view an individual variable's metadata, and we'll print out the list of variables for our dataset. 

Next, print out a simple list of all associated variable names by looping the same variable response we submitted above, this time for each variable:
```

34. Retrieve associated variable IDs from the collection
1. Set desired (different) output format for variables
1. Format a query to fetch the service for each variable ID (why isn't this a link relation?)
1. Perform CMR variable search for each variable ID
1. Parse the search result
1. Retrieve variable from the search result
1. Retrieve variable names from the variable

##### Notes
* Assumes we could reuse the already-searched and parsed collection record from service lookup step with refactoring


In [18]:
%time 

variables = response['feed']['entry'][0]['associations']['variables']
print(variables)

var_url = "https://cmr.earthdata.nasa.gov/search/variables"
var_response = requests.get(var_url, 
                        params={
                            'concept_id': variables[0],
                            },
                        headers={
                            'Accept': 'application/vnd.nasa.cmr.umm_results+json'
                            }
                       )
var_response = var_response.json()
#pprint(var_response)

var_list = []
for i in range(len(variables)):
    var_response = requests.get(var_url, 
                            params={
                                'concept_id': variables[i],
                                },
                            headers={
                                'Accept': 'application/vnd.nasa.cmr.umm_results+json'
                                }
                           )
    var_response = var_response.json()
    var_list.append(var_response['items'][0]['umm']['Name'])

pprint(var_list)

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 6.68 µs
['V1997812737-POCLOUD', 'V1997812697-POCLOUD', 'V2112014688-POCLOUD', 'V1997812756-POCLOUD', 'V1997812688-POCLOUD', 'V1997812670-POCLOUD', 'V1997812724-POCLOUD', 'V2112014684-POCLOUD', 'V1997812701-POCLOUD', 'V1997812681-POCLOUD', 'V2112014686-POCLOUD', 'V1997812663-POCLOUD', 'V1997812676-POCLOUD', 'V1997812744-POCLOUD', 'V1997812714-POCLOUD']
['sses_standard_deviation_4um',
 'l2p_flags',
 'time',
 'dt_analysis',
 'sses_standard_deviation',
 'sst_dtime',
 'sses_bias_4um',
 'lat',
 'sea_surface_temperature_4um',
 'sses_bias',
 'lon',
 'sea_surface_temperature',
 'quality_level',
 'wind_speed',
 'quality_level_4um']


### 3. Request variable subset


#### Steps Involved (NEED TO UPDATE):

'''
We can check on the progress of a processing job with status(). This method blocks while communicating with the server but returns quickly.

Depending on the size of the request, it may be helpful to wait until the request has completed processing before the remainder of the code is executed. The wait_for_processing() method will block subsequent lines of code while optionally showing a progress bar.
'''

41. Create Harmony client with previously obtained auth (or set up / use .netrc)
1. Construct Harmony request object using desired spatial / temporal / variables
1. Submit the Harmony request, obtaining a Job ID

In [25]:
%time

harmony_client = Client()

request = Request(
    collection=Collection(id=short_name),
    spatial=BBox(-97.77667,21.20806,-83.05197,30.16605),
    temporal={
        'start': dt.datetime(2021, 8, 20),
        'stop': dt.datetime(2021, 8, 21),
    },
)

job_id = harmony_client.submit(request)
print(job_id)

harmony_client.status(job_id)

harmony_client.wait_for_processing(job_id, show_progress=True)

CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 5.25 µs
dfbc163f-5815-4f08-ae0c-2b6b4870df91


 [ Processing: 100% ] |###################################################| [|]


## STAC workflow (in progress)

### 2. Declare time and bounding box search parameters

_Using a bounding box that will cover granules supported in on-prem OPeNDAP and L2SS_

#### Steps Involved:

8. Format desired spatial bounding box into CMR query format
1. Format desired temporal range into CMR query format
1. (Non-Code) Determine precise data product short name (perhaps already known by users)
1. Set data product short name in code
1. (Non-Code) Determine precise data product version identifier, including leading zeroes
1. Set data product version identifier in code
1. (Non-Code) Determine data provider identifier for the collection owner
1. Set data provider identifier in code

##### Alternative for 12-17

10. (Non-Code) Go to Earthdata Search
1. (Non-Code) Search for data product short name or similar
1. (Non-Code) Select correct collection, revealing CMR Concept ID
1. Copy/paste the CMR Concept ID into code
